In [ ]:
import jieba
import pandas as pd
import numpy as np
from PIL import Image
import collections
import matplotlib.pyplot as plt
import wordcloud
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import
import pyLDAvis.sklearn
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)#解决引入pyLDAvis的报错问题 https://github.com/bmabey/pyLDAvis/issues/196

In [ ]:
goods_list=[line.strip() for line in open('goods_list.txt', encoding="utf-8").readlines()]

stopword = [line.strip() for line in open('stopword.txt', encoding="utf-8").readlines()]
stopword.append(' ')
stopword.append('\n')

## 分词+去停用词

In [ ]:
def seg_sentence(sentence,stopwords):
    sentence_seged = jieba.cut(sentence.strip())
    # loading stop word list
    re = []
    for word in sentence_seged:
        if word not in stopwords:
            re.append(word)
    return re

def seg_comment(goods):
    f= open(goods+'/comment.txt','r')
    comment=f.read()
#     for i in range(1,999):
#         comments+=f.readline()
    f.close()
    segs = seg_sentence(comment,stopword)
    return segs

In [ ]:
segs_dic={}
for goods in goods_list:
    segs_dic[goods]=seg_comment(goods)
    
for (k,v) in segs_dic.items():
    f=open(k+'/segs.txt','w')
    for seg in v:
        f.write(seg+' ')
    f.close()

In [ ]:
print(len(segs_dic))
i=0
for (k,v) in segs_dic.items():
    print(len(v))
    i+=len(v)
print(i)

In [ ]:
# for (k,v) in segs_dic.items():
#     f=open(k+'/segs.txt','a')
#     for seg in v:
#         f.write(seg+' ')
#     f.close()
# 仅在第一次运行时进行分词后文本的保存

In [ ]:
segs_dic={}
for goods in goods_list:
    segs_dic[goods]=open(goods+'/segs.txt').readline().split(' ')
# print(len(segs_dic))
# for (k,v) in segs_dic.items():
#     print(len(v))
#     print(v[0:7])

## 生成词云

In [ ]:
font='/System/Library/Fonts/Hiragino Sans GB.ttc'
background_img='background.jpg'
mask = np.array(Image.open(background_img).convert('RGB'))

In [ ]:
def segs_to_wc(goods,segs,font='Hiragino Sans GB.ttc',bg='background.jpg'):
    mask = np.array(Image.open(bg).convert('RGB'))
    
    wc = wordcloud.WordCloud(
                font_path=font,  # 设置字体格式
                mask = mask,    # 设置背景图
                max_words=400,     # 最多显示词数
                max_font_size=80,  # 字体最大值
                scale=16,           # 调整图片清晰度，值越大越清楚
                background_color = "white"
    )

    frequency = collections.Counter(segs)
    
    wc.generate_from_frequencies(frequency)         # 从字典生成词云
    image_colors = wordcloud.ImageColorGenerator(mask)  # 从背景图建立颜色方案
    wc.recolor(color_func=image_colors)             # 将词云颜色设置为背景图方案

    # plt.imshow(wc,interpolation='bilinear')
    # plt.axis("off")
    # plt.show()

    wc.to_file(goods+'/wc.png')

In [ ]:
for (k,v) in segs_dic.items():
    segs_to_wc(k,v)

## 以下写主题分析与词频统计

In [ ]:
def save_top_words(model, feature_names, n_top_words,goods):
    f=open(goods+'/topic.txt','w')
    for topic_idx, topic in enumerate(model.components_):
        f.write("Topic #%d:" % (topic_idx+1))
        f.write('\n')
        f.write(" ".join([feature_names[i]
            for i in topic.argsort()[:-n_top_words - 1:-1]]))
        f.write('\n')
    f.close()

def topic_anaz(goods,segs):
    n_features = 1000
    tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                    max_features=n_features,
                                    max_df = 0.5,
                                    min_df = 10)
    tf = tf_vectorizer.fit_transform(pd.array(segs))
    
    n_topics = 3
    lda = LatentDirichletAllocation(n_components=n_topics,
                                    max_iter=5,
                                    learning_method='online',
                                    learning_offset=50.,
                                    random_state=0)
    
    lda.fit(tf)#运行时间很长
    
    n_top_words = 20
    tf_feature_names = tf_vectorizer.get_feature_names()
    save_top_words(lda, tf_feature_names, n_top_words,goods)
    
    pyLDAvis.enable_notebook()
    LDAvis_prepared=pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer)
    pyLDAvis.save_html(LDAvis_prepared,goods+'/topic.html')

In [ ]:
import time

time_start=time.time()

for (k,v) in segs_dic.items():
    topic_anaz(k,v)
    
time_end=time.time()
print('time cost',time_end-time_start,'s')